In [1]:
import cv2
import tkinter as tk
import tkinter.filedialog as tkfd
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
%matplotlib inline

Using TensorFlow backend.


In [2]:
def load_images(folder):
        image_list=[]
        for filename in os.listdir(folder):
            img=cv2.imread(os.path.join(folder,filename))
            if img is not None:
                image_list.append(img)
        return image_list

In [32]:
tk.Tk().withdraw()
path_Train=tkfd.askdirectory()

In [4]:
X_Train=load_images(path_Train)
X_Train=np.array(X_Train)

In [5]:
tk.Tk().withdraw()
path_Test=tkfd.askdirectory()

In [6]:
X_Test=load_images(path_Test)
X_Test=np.array(X_Test)

In [7]:
tk.Tk().withdraw()
path_Validation=tkfd.askdirectory()

In [8]:
X_Validation=load_images(path_Validation)
X_Validation=np.array(X_Validation)

In [9]:
os.chdir(path=tkfd.askdirectory())
data_Train=pd.read_csv("ISIC-2017_Training_Part3_GroundTruth.csv")
data_Test=pd.read_csv("ISIC-2017_Test_v2_Part3_GroundTruth.csv")
data_Validation=pd.read_csv("ISIC-2017_Validation_Part3_GroundTruth.csv")

In [10]:
data_Train=data_Train.iloc[0:2000,1]
data_Test=data_Test.iloc[0:600,1]

y_Train=data_Train
y_Test=data_Test

y_Train=np.array(y_Train)
y_Test=np.array(y_Test)

In [11]:
y_Train = np_utils.to_categorical(data_Train)
y_Test = np_utils.to_categorical(data_Test)

In [31]:
learning_rate=0.01
training_iterations=50000
batch_size=100
display_step=10
logs_path = tkfd.askdirectory()
n_input=64*64*3 # Data input 
n_classes=2 # Classes
dropout=0.75

x=tf.placeholder("float",[None,n_input])
y=tf.placeholder("float",[None,n_classes])
Drop_prob=tf.placeholder(tf.float32)

In [20]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 64, 64, 3])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [15]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([3, 3, 3, 16])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([3, 3, 16, 16])),
    # fully connected, 16*16*64 inputs, 4096 outputs
    'wd1': tf.Variable(tf.random_normal([16*16*16, 12288])),
    # 4096 inputs, 3 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([12288, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([16])),
    'bc2': tf.Variable(tf.random_normal([16])),
    'bd1': tf.Variable(tf.random_normal([12288])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
with tf.name_scope("Model"):
    pred= conv_net(x, weights, biases, Drop_prob)

# Define loss and optimizer
with tf.name_scope("loss"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
with tf.name_scope("Adam"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
with tf.name_scope("Accuracy"):
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# summaries
tf.summary.scalar("loss",cost)
tf.summary.scalar("accuracy",accuracy)
merged_summary_op=tf.summary.merge_all()

In [28]:
# launch the graph
with tf.Session() as sess:
    sess.run(init)
    step=1
    # Training 
    summary_writer=tf.summary.FileWriter(logdir=logs_path,graph=tf.get_default_graph())
    while step*batch_size<training_iterations:
        train_idx=np.random.randint(X_Train.shape[0],size=batch_size)
        batch_x_Train=X_Train[train_idx,:]
        batch_y_Train=y_Train[train_idx]
        _,c,summary=sess.run([optimizer,cost,merged_summary_op],feed_dict={x:batch_x_Train,y:batch_y_Train,Drop_prob:dropout})
        summary_writer.add_summary(summary,step*batch_size)
        if step%display_step==0:
            loss,acc=sess.run([cost,accuracy],feed_dict={x:batch_x_Train,y:batch_y_Train,Drop_prob:1.})
        
            print("iters:"+ str(step*batch_size)+",Minibatch Loss=" + \
                "{:.6f}".format(loss)+",Training Accuracy="+\
                "{:.5f}".format(acc))
        step+=1
    print("Optimization Finished")
    test_idx=np.random.randint(X_Test.shape[0],size=batch_size)
    batch_x_Test=X_Test[test_idx,:]
    batch_y_Test=y_Test[test_idx]
    print("Testing accuracy",\
        sess.run(accuracy,feed_dict={x:batch_x_Test,y:batch_y_Test,Drop_prob:1.}))

ValueError: Cannot feed value of shape (100, 64, 64, 3) for Tensor 'Placeholder_11:0', which has shape '(?, 12288)'

In [29]:
X_Train.shape

(2000, 64, 64, 3)